In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import utils
from datasets import CommentsDataset, CommentTagsDataset, NLIDataset
from models import BERTCls, SimplePerceptron

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification, BertModel, BertForNextSentencePrediction, AutoModelForSequenceClassification

import torch

from torch import nn
from torch.utils.data import DataLoader

TAGS = [
    'ASSORTMENT',
    'CATALOG_NAVIGATION',
    'DELIVERY',
    'PAYMENT',
    'PRICE',
    'PRODUCTS_QUALITY',
    'PROMOTIONS',
    'SUPPORT',
]

CLASSES = [f'trend_id_res{i}' for i in range(50)]

CLASSES2TRENDS = {0: 'Долгая доставка',
 1: 'Доставка стала долгой',
 2: 'Время доставки не соответствует заявленому',
 3: 'Регулярные опоздания',
 4: 'Не отследить реальное время доставки',
 5: 'Курьер на карте',
 6: 'Нет доставки по адресу',
 7: 'Не предупреждаем об удалении товара',
 8: 'Высокая минимальная сумма заказа',
 9: 'Сумма заказа меняется во время набора корзины',
 10: 'Минимальная сумма заказа',
 11: 'Товары с подходящим сроком годности',
 12: 'Высокие цены',
 13: 'Не довезли товар',
 14: 'Товар испорчен во время доставки',
 15: 'Просроченные товары',
 16: 'Замечания по работе курьеров',
 17: 'Не читаем комментарии',
 18: 'Спасибо',
 19: 'Нет смысла',
 20: 'Всё нормально',
 21: 'Всё плохо',
 22: 'Скидки для постоянных клиентов',
 23: 'Больше акций/скидок',
 24: 'Скидка/промокод распространяется не на все товары',
 25: 'Непонятно как работает скидка',
 26: 'Не сработала скидка/акция/промокод',
 27: 'Качество товаров',
 28: 'Маленький ассортимент',
 29: 'Нет в наличии товара',
 30: 'Качество поддержки',
 31: 'Замечания по работе сборщика',
 32: 'Отменили заказ',
 33: 'Знание русского языка',
 34: 'Привезли чужой заказ',
 35: 'Долго на сборке',
 36: 'Сравнивают с конкурентами',
 37: 'Скидки за опоздание',
 38: 'Курьеры отменяют заказ',
 39: 'Не тянет на тенденцию',
 40: 'Испорченные товары',
 41: 'Не нравится интерфейс приложения',
 42: 'Приложение зависает',
 43: 'Быстрая доставка',
 44: 'Условия работы курьеров',
 45: 'СберСпасибо',
 46: 'Время работы',
 47: 'Неудобный поиск',
 48: 'Платежи',
 49: 'Возврат денег'}
CLASSES2FULLTRENDS = {
    0: 'Долгая доставка, долго везёте. Долго. Приезжает через час. Дольше часа. Опоздания. Время. Слишком долгая доставка. Убрали быструю доставку. Задержки. Неторопливо. Не дождаться. Жду. Доставляют не вовремя',
    1: 'Доставка стала долгой, раньше доставляли быстрее. Раньше доставка была быстрая. Стали доставлять от [NUM] минут. Стало дольше. Было быстро. Сейчас долго. Скорость доставки ухудшилась',
    2: 'Не соответствует заявленому времени доставки. Дольше чем указано. Пишите [NUM] мин, а везёте [NUM] мин. Обещания в [NUM]м. Заявленных [NUM] минут доставку. Дольше, чем обещали. Обман с доставкой, доставка за [NUM] мин',
    3: 'Регулярные опоздания, каждый заказ привозят с опозданием, доставка постоянно задерживается. Частые опоздания. Иногда задерживается. Каждый раз жду. Каждый раз не вовремя. Переодически очень долго везут заказ.',
    4: 'Не отследить реальное время доставки, невозможно понять, когда приедет заказ. Неудобно следить за доставкой. Нет понимания точного времени прибытия курьера. Время ожидания не соответствует',
    5: 'Курьер на карте, отображение курьера на карте, курьера на карте не видно, отслеживать курьера. Когда будет заказ?',
    6: 'Нет доставки по адресу, границы доставки, не везде доставляем',
    7: 'Не предупреждаете об удалении товара, нет замену товара, товар удалили из заказа, убрали из заказа. На этапе сборки. Не предупредили об отмене. Нет возможности заменить товар. Отмена товара.',
    8: 'Высокая минимальная сумма заказа, дорогая доставка, заказ от большой суммы',
    9: 'Сумма заказа меняется во время набора корзины, приходится несколько раз докладывать товары до минимальной суммы',
    10: 'Минимальная сумма заказа, наличие минимальной определенной стоимости, меняется минимальная сумма. Заказ от [NUM] рублей',
    11: 'Товары с подходящим сроком годности, привозим товары с практически истекшим сроком годности, конец срока годности', 
    12: 'Высокие цены, дорого, цена отличается от других. Необоснованные цены. Неудобная стоимость',
    13: 'Не довезли товар, забыли положить, не доложили. Не доставили заказ. Не хватает продуктов',
    14: 'Товар испорчен во время доставки: побили яйца, разлился йогрут, порваная упаковка, разлившееся, раздавленные. Повреждения во время доставки',
    15: 'Просроченные продукты, привезли товар с истекшим сроком годности, просрочка. Несвежие товары',
    16: 'Замечания по работе курьеров. Курьеры хамят, не читают комментарии, не понимают русский язык, не доносят заказ до двери. Не компетентные, безответственные курьеры',
    17: 'Не читаем комментарии, не выполнили комментарий. Игнорируем замечание к заказу',
    18: 'Спасибо, молодцы так деражть, 👍🏻, ❤️. Благодарю сервис. Отлично, хороший сервис. Классно, супер. Удобно, здорово, быстро. Ок, пойдет. Норм',
    19: 'Нет смысла, не ясен смысл. ?, . хз, не знаю, ок, хуй ня, гавно, бубу, охуели, ???',
    20: 'Всё нормально. Все норм, нормально, устраивает, неплохо. Пойдет, ничего, ок. Все хорошо',
    21: 'Всё плохо. Плохой сервис, все хуево. Хуета. Портитесь. Ничего хорошего, отвратительно, хуже некуда',
    22: 'Нет скидок для постоянных клиентов, скидок на следующий заказ, программы лояльности. Персональные скидки, акции, промоакции, бонусы, промокоды, специальные предложения',
    23: 'Больше акций/скидок/промокодов. Не хватает скидок на всю корзину, акцию "товар за 1 рубль". Мало акций/скидок',
    24: 'Скидка/промокод распространяется не на все товары. Не применяются скидки, промокоды. Не применяется корзинная скидка, корзинная скидка применяется только к какой-то категории товаров',
    25: 'Непонятно как работает скидка, странно работает скидка, не понятно почему скидка применилась не ко всем товарам, не понятно к каким товарам применилась скидка',
    26: 'Не сработала скидка/акция/промокод. Не работает скидка. Не работает промокод. Не действует промокод. Списались баллы',
    27: 'Качество товаров. Некачественные товары. Невкусно. Несвежее, вялое, неспелое. Есть невозможно. Испорчен',
    28: 'Маленький ассортимент, маленький выбор товаров. Ассортимент расстраивает. Нехватка, отсутствует товар, мало товаров. Мало продуктов. Расширьте ассортимент.',
    29: 'Нет в наличии товара, товары быстро заканчиваются, нет завоза. Ничего нет вечером, уведомление о поступлении, поступит ли товар снова. Товар раскупили',
    30: 'Качество поддержки. Медленно отвечаем, бот не переключает на оператора, закрываем обращения не разобравшись. Поддержка плохая. Молчат, не отвечают',
    31: 'Сборщик. Сборщик не положил. Забыли положить. Несовместимые продукты в один пакет, термопакеты для замороженных продуктов, кладём не то, путаем товары. Плохая сборка продуктов',
    32: 'Отменили заказ, не довозят заказ. Заказ отменён',
    33: 'Знание русского языка, курьеры не понимают/не читают/не говорят по-русски, хачи, узбеки. Ни слова ни понимают. Не русские. Не умеют читать. Грязные представители южных стран',
    34: 'Привезли чужой заказ, перепутали заказ. Путаница с заказами. Привозят не твой заказ. Не мой заказ',
    35: 'Сборка заказа. Долго на сборке, долго собираем заказ. Заказ ещё не передали. Ждём, а курьер ещё не вышел',
    36: 'Сравнивают с конкурентами, сравнение с другими сервисами доставки продуктов. Проигрывает пяторчке. Сбермаркет выгоднее. Яндекс и деливери лучше. Яндекс еда. Лавка, озон фреш. Вкусвилл быстрее',
    37: 'Скидки за опоздание, скидки/компенсацию в качестве извинения, не компенсирует. Компенсация. Бонусы за задержку. Скидка на след заказ',
    38: 'Курьеры отменяют заказ, курьер молча отменяет заказ. Он уже отменил заказ',
    39: 'Не тянет на тенденцию', # будем ставить только когда все 0
    40: 'Испорченные товары, срок годности, тухлое, гнилые, пропавшая, плесень',
    41: 'Не нравится интерфейс приложения, не нравится приложение. Неудобный дизайн, расположение категорий. Неинтуитивное непонятное управление. Неудобно пользоваться. Добавьте раздел, редактировать заказ.',
    42: 'Приложение зависает, приложение работает не стабильно, зависает. Приложение тупит, глючит. Тормозит. Андроид фризит, айос.',
    43: 'Быстрая доставка, быстро доставили, скорость доставки',
    44: 'Условия работы курьеров, повысить ставку курьерам, одеть потеплее. Плохое отношение к сотрудникам. Ужасное обращение с сотрудниками. Больше внимания к курьерам и их условиям. Перерыв работникам',
    45: 'СберСпасибо, сложности при подключении, периодически не оплатить бонусами. Бонусы спасибо. Бонусы Сбербанкспасибо. Баллы Сбер Спасибо.',
    46: 'Время работы, рано закрываетесь, круглосуточная доставка, сделайте доставку с [NUM] часов. Работаете с [NUM] часов утра. Не работаете после [NUM] часов',
    47: 'Неудобный поиск, не нравится навигация по каталогу, не нравится структура каталога. Сторисы в каталоге, акции среди товаров. Не хватает фильтров для поиска. Поиск по приложению',
    48: 'Платежи, жалобы на проведение платежей, оплата курьеру наличными или картой, проблемы с оплатой, СБП, отображение карты, не проходят платежи, чеки, квитанции',
    49: 'Возврат денег, долгие возвраты. Не вернули деньги. Не возвращают деньги. Не вернули деньги за прошлый заказ'
}
device = 'cuda:0'

### BERT - best model recreation

In [2]:
model_name = 'ai-forever/ruElectra-large'
model_name = 'ai-forever/ruRoberta-large'
# model_name = 'deepvk/deberta-v1-base'
model_name = 'ai-forever/rubert-tiny2'
model_name = 'cointegrated/rubert-base-cased-nli-twoway'
model_name = 'ai-forever/ruBert-large'
model = BERTCls(AutoModel.from_pretrained(model_name))
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
df = pd.read_csv('data/train.csv')
df = utils.prepare_data(df)
# df['tags'] = df['tags'].replace(float('nan'), '{NONE}')
df

,index,assessment,tags,text,SUPPORT,CATALOG_NAVIGATION,ASSORTMENT,PROMOTIONS,PAYMENT,DELIVERY,...,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49,NONE
0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,NaN
1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,NaN
3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4618,26325,2.0,{PRODUCTS_QUALITY},Привезли прокисший суп,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4619,661,6.0,{DELIVERY},пойдет,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
4620,1870,6.0,"{PROMOTIONS,PRICE,SUPPORT,PRODUCTS_QUALITY}",Не дают абузить поддержка не возвращает деньги...,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4621,22650,2.0,"{DELIVERY,PRODUCTS_QUALITY}","Очень плохая доставка в первую очередь, постоя...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN


In [4]:
test_df = pd.read_csv('data/test.csv')
test_df = utils.prepare_data(test_df)
test_df

,index,assessment,tags,text,SUPPORT,CATALOG_NAVIGATION,ASSORTMENT,PROMOTIONS,PAYMENT,DELIVERY,PRICE,PRODUCTS_QUALITY,NONE
0,3135,3.0,{DELIVERY},"Последнее время думаю плохо, сроки доставки да...",0,0,0,0,0,1,0,0,NaN
1,4655,2.0,"{PRICE,DELIVERY,ASSORTMENT}",Цены намного выше магазинных но радуют акции,0,0,1,0,0,1,1,0,NaN
2,22118,2.0,"{CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}","Доставка за [NUM] минут, заказ даже не начали ...",0,1,1,0,0,1,0,0,NaN
3,23511,0.0,{DELIVERY},Ужасно долгая доставка,0,0,0,0,0,1,0,0,NaN
4,45,6.0,"{ASSORTMENT,PROMOTIONS}",Добрый вечер! Вы большие молодцы. Меня всё уст...,0,0,1,1,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9010,3523,3.0,"{PRICE,SUPPORT,DELIVERY}",Задержка с доставкой не даете промокод на скид...,1,0,0,0,0,1,1,0,NaN
9011,24925,6.0,"{PRICE,PRODUCTS_QUALITY,ASSORTMENT}",Очень удобный формат сервиса и очень маленький...,0,0,1,0,0,0,1,1,NaN
9012,6327,6.0,"{PAYMENT,ASSORTMENT,DELIVERY}","Сумма заказа почти всегда высокая, что зачасту...",0,0,1,0,1,1,0,0,NaN
9013,530,3.0,"{PRODUCTS_QUALITY,SUPPORT,DELIVERY}","Часто, заказываю у вас молочную продукцию, при...",1,0,0,0,0,1,0,1,NaN


In [5]:
EPOCHS = 35
BATCH_SIZE = 20
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


train_data = CommentsDataset(train_df, tokenizer=tokenizer)
val_data = CommentsDataset(val_df, tokenizer=tokenizer)
test_data = CommentsDataset(test_df, tokenizer=tokenizer, is_test=True)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=128)
test_dataloader = DataLoader(test_data, batch_size=128)
val_data=[x['label'] for x in list(val_data)]

# for param in model.backbone.bert.parameters():
#     param.requires_grad = False
    
model = model.to(device)
optimizer = torch.optim.Adam(
    # model.parameters(), lr=5e-5
    [
        {"params": model.backbone.parameters(), "lr": 5e-5},
        {"params": model.linear_layer_with_tag.parameters(), "lr": 5e-4}
    ]
)
#y_true = [x['label'] for x in list(val_data)]
# weight = torch.tensor(np.sum(np.array(y_true), axis=0, keepdims=True) / np.sum(np.array(y_true)))
# weight = weight.to(device)
weight=None
loss_fn = nn.BCEWithLogitsLoss()

In [7]:
# for param in model.backbone.parameters():
#     param.requires_grad = False
# optimizer = torch.optim.AdamW(
#     [
#         {"params": model.backbone.bert.parameters(), "lr": 5e-4},
#         {"params": model.linear_without_tag.parameters(), "lr": 1e-3}
#     ]
# )

In [9]:
# 12 epochs
losses = utils.train(
    model, train_dataloader, optimizer, loss_fn, 
    6, val_dataloader=val_dataloader, 
    val_data=val_data
)
plt.plot(losses)
optimizer.param_groups[0]['lr'] = 6e-6
optimizer.param_groups[1]['lr'] = 5e-5
losses = utils.train(
    model, train_dataloader, optimizer, loss_fn, 
    2, val_dataloader=val_dataloader, 
    val_data=val_data
)
plt.plot(losses)

100%|██████████| 8/8 [00:11<00:00,  1.44s/it]


threshold: 0.5
accuracy: 0.5654054054054054


100%|██████████| 8/8 [00:11<00:00,  1.44s/it]


threshold: 0.5
accuracy: 0.5718918918918919


  0%|          | 1/232 [00:00<02:21,  1.64it/s]


KeyboardInterrupt: 

In [10]:
torch.save(model.state_dict(), 'models/ai-forever/ruBert-large_6+2ep_6e-6')

In [11]:
preds = torch.tensor(utils.eval(model, val_dataloader))
#print(preds)
prev_eval = 0
for thresh in np.arange(0.01, 0.99, 0.01):
    print(f'threshold: {thresh}')    
    preds_thresh = (nn.functional.sigmoid(preds) > thresh).int()
    eval_acc = accuracy_score(val_data, preds_thresh.tolist())
    # if eval_acc < prev_eval:
    #     break
    prev_eval = eval_acc
    print(f'accuracy: {eval_acc}')

100%|██████████| 8/8 [00:11<00:00,  1.41s/it]


threshold: 0.01
accuracy: 0.1891891891891892
threshold: 0.02
accuracy: 0.26594594594594595
threshold: 0.03
accuracy: 0.30486486486486486
threshold: 0.04
accuracy: 0.3491891891891892
threshold: 0.05
accuracy: 0.3783783783783784
threshold: 0.060000000000000005
accuracy: 0.4
threshold: 0.06999999999999999
accuracy: 0.412972972972973
threshold: 0.08
accuracy: 0.4227027027027027
threshold: 0.09
accuracy: 0.4259459459459459
threshold: 0.09999999999999999
accuracy: 0.44432432432432434
threshold: 0.11
accuracy: 0.45297297297297295
threshold: 0.12
accuracy: 0.46378378378378377
threshold: 0.13
accuracy: 0.46702702702702703
threshold: 0.14
accuracy: 0.47783783783783784
threshold: 0.15000000000000002
accuracy: 0.4864864864864865
threshold: 0.16
accuracy: 0.492972972972973
threshold: 0.17
accuracy: 0.5070270270270271
threshold: 0.18000000000000002
accuracy: 0.5124324324324324
threshold: 0.19
accuracy: 0.5156756756756756
threshold: 0.2
accuracy: 0.5232432432432432
threshold: 0.21000000000000002
accu

In [12]:
preds = torch.tensor(utils.eval(model, test_dataloader))
classes = utils.get_classes((nn.functional.sigmoid(preds) > 0.47).int())
test_df['target'] = classes
test_df

100%|██████████| 71/71 [01:53<00:00,  1.59s/it]


,index,assessment,tags,text,SUPPORT,CATALOG_NAVIGATION,ASSORTMENT,PROMOTIONS,PAYMENT,DELIVERY,PRICE,PRODUCTS_QUALITY,NONE,target
0,3135,3.0,{DELIVERY},"Последнее время думаю плохо, сроки доставки да...",0,0,0,0,0,1,0,0,NaN,1 2 12
1,4655,2.0,"{PRICE,DELIVERY,ASSORTMENT}",Цены намного выше магазинных но радуют акции,0,0,1,0,0,1,1,0,NaN,12
2,22118,2.0,"{CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}","Доставка за [NUM] минут, заказ даже не начали ...",0,1,1,0,0,1,0,0,NaN,2 35
3,23511,0.0,{DELIVERY},Ужасно долгая доставка,0,0,0,0,0,1,0,0,NaN,0
4,45,6.0,"{ASSORTMENT,PROMOTIONS}",Добрый вечер! Вы большие молодцы. Меня всё уст...,0,0,1,1,0,0,0,0,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9010,3523,3.0,"{PRICE,SUPPORT,DELIVERY}",Задержка с доставкой не даете промокод на скид...,1,0,0,0,0,1,1,0,NaN,37
9011,24925,6.0,"{PRICE,PRODUCTS_QUALITY,ASSORTMENT}",Очень удобный формат сервиса и очень маленький...,0,0,1,0,0,0,1,1,NaN,12 23 28
9012,6327,6.0,"{PAYMENT,ASSORTMENT,DELIVERY}","Сумма заказа почти всегда высокая, что зачасту...",0,0,1,0,1,1,0,0,NaN,8
9013,530,3.0,"{PRODUCTS_QUALITY,SUPPORT,DELIVERY}","Часто, заказываю у вас молочную продукцию, при...",1,0,0,0,0,1,0,1,NaN,15


In [22]:
test_df[test_df['target'] == ''].iloc[56]['text']

'Сумма заказа пишет от [NUM], делаю заказ пишет заказ от [NUM] уже...ужасно работает приложение самокат'

In [13]:
test_df[['index', 'target']].to_csv('submissions/submission_0.47sigmbelarge.csv', index=False)